# **AI Self-Portrait Generator Using Twitter and Stable Diffusion**
This notebook allows you to:
- Collect Twitter replies to a specific tweet.
- Analyze those replies to generate a text description of a visual appearance.
- Use the description to create an AI-generated self-portrait using Stable Diffusion.
- Post the image back to Twitter with a caption.

### **Requirements Before Starting:**
1. **Twitter API Keys:** You need access to the Twitter API. Follow this [guide](https://developer.twitter.com/en/docs) to get your keys.
2. **HuggingFace Token:** Get a free token from [HuggingFace](https://huggingface.co/settings/tokens) to use the Stable Diffusion model.
3. **Google Colab Account:** This notebook runs entirely in Colab.

In [ ]:
# Install required libraries
!pip install tweepy transformers diffusers huggingface_hub schedule spacy
!python -m spacy download en_core_web_sm

### Step 1: Twitter API Authentication
Input your Twitter API keys below.

In [ ]:
import tweepy

# Input your Twitter API credentials
API_KEY = 'your_api_key'
API_SECRET = 'your_api_secret'
ACCESS_TOKEN = 'your_access_token'
ACCESS_SECRET = 'your_access_secret'

# Authenticate
auth = tweepy.OAuth1UserHandler(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)
print("Authenticated with Twitter API!")

### Step 2: Fetch Twitter Replies
Provide the Tweet ID of the post you want to analyze.

In [ ]:
def fetch_replies(tweet_id, username):
    replies = []
    for tweet in tweepy.Cursor(api.search_tweets, q=f"to:{username}", since_id=tweet_id, tweet_mode='extended').items():
        if tweet.in_reply_to_status_id == tweet_id:
            replies.append(tweet.full_text)
    return replies

# Input Tweet ID and username
tweet_id = 29952163  # Replace with your Tweet ID
username = '@What_ilooklike'  # Replace with your Twitter username
replies = fetch_replies(tweet_id, username)
print("Fetched replies:", replies)

### Step 3: Process Replies
Extract appearance-related details from the replies using NLP.

In [ ]:
from transformers import pipeline
import spacy

# Load models
summarizer = pipeline("summarization")
nlp = spacy.load("en_core_web_sm")

# Process Replies
def process_replies(replies):
    descriptions = []
    for reply in replies:
        doc = nlp(reply)
        for token in doc:
            if token.pos_ == 'ADJ':
                descriptions.append(token.text)
    # Summarize output
    summary = summarizer(description_text, max_length=50, min_length=10, do_sample=False)
    return summary[0]["summary_text"]

attributes = process_replies(replies)
print("Extracted attributes:", attributes)

### Step 4: Generate Image
Use Stable Diffusion to generate an AI-generated image based on the descriptions.

In [ ]:
from diffusers import StableDiffusionPipeline

# Load Stable Diffusion
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("cuda")  # Use GPU for faster processing

def generate_image(description, output_path="generated_image.png"):
    image = pipe(description).images[0]
    image.save(output_path)
    return output_path

image_path = generate_image(attributes)
print("Generated image saved at:", image_path)

### Step 5: Post Image to Twitter
Share the generated image on Twitter.

In [ ]:
def post_image(image_path, caption):
    api.update_status_with_media(filename=image_path, status=caption)

post_image(image_path, "Here’s what I look like today based on your replies!")